In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
dataset = pd.read_csv('creditcard.csv')

In [2]:
dataset['new_amount'] = StandardScaler().fit_transform(dataset['Amount'].values.reshape(-1, 1))
dataset.drop(['Time', 'Amount'], axis=1, inplace=True)
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,new_amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [3]:
train_set_percentage = 0.5
################################################### 
fraud_series = dataset[dataset['Class'] == 1]
idx = fraud_series.index.values
np.random.shuffle(idx)
fraud_series.drop(idx[:int(idx.shape[0]*train_set_percentage)], inplace=True)
dataset.drop(fraud_series.index.values, inplace=True)
###################################################

C:\Users\HARSH\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
###################################################
# normal dataset with the same size of the fraud_series (training dataset)
normal_series = dataset[dataset['Class'] == 0] 
idx = normal_series.index.values
np.random.shuffle(idx)
normal_series.drop(idx[fraud_series.shape[0]:], inplace=True)
dataset.drop(normal_series.index.values, inplace=True)
###################################################

C:\Users\HARSH\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# build the training dataset
new_dataset = pd.concat([normal_series, fraud_series])
new_dataset.reset_index(inplace=True, drop=True)
y = new_dataset['Class'].values.reshape(-1, 1)
new_dataset.drop(['Class'], axis=1, inplace=True)
X = new_dataset

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
# Attributes that will be used by the gridsearchCV algorithm 
attr={'C': [0.1, 1, 2, 5, 10, 25, 50, 100],
      'gamma': [1e-1, 1e-2, 1e-3]
     }

X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.3, random_state=10)

model = SVC(probability=True)
classif = GridSearchCV(model, attr, cv=5)
classif.fit(X_train, y_train)
y_pred_svm = classif.predict(X_test)
y_prob_svm = classif.predict_proba(X_test)
print('Accuracy: ',accuracy_score(y_pred_svm, y_test))

Accuracy:  0.972972972973


In [7]:
knn = KNeighborsClassifier(n_neighbors = 5,n_jobs=16)        # n_jobs The number of jobs to run in parallel for both fit and predict. None means 1 unless in a joblib.parallel_backend context. -1 means using all processors.
knn.fit(X_train,y_train)                                     # n_neighbors Number of neighbors to use by default for kneighbors queries.
print("classifier created")
y_pred_knn = knn.predict(X_test)
y_prob_knn = knn.predict_proba(X_test)
print('Accuracy: ',accuracy_score(y_pred_knn, y_test))
#score = knn.score(X_test,y_test)
#print("model evaluated")
#print(score)

classifier created
Accuracy:  0.959459459459


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

gcf = GaussianNB()
gcf.fit(X_train, y_train)
y_pred_gcf = gcf.predict(X_test)
y_prob_gcf = gcf.predict_proba(X_test)
print('Accuracy: ',accuracy_score(y_pred_gcf, y_test))

Accuracy:  0.918918918919


In [23]:
y_pred_gcf

array([1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1], dtype=int64)

In [24]:
y_prob_gcf

array([[  1.25391689e-275,   1.00000000e+000],
       [  9.58241414e-243,   1.00000000e+000],
       [  1.00000000e+000,   1.49304419e-016],
       [  0.00000000e+000,   1.00000000e+000],
       [  2.26619304e-031,   1.00000000e+000],
       [  0.00000000e+000,   1.00000000e+000],
       [  1.00000000e+000,   4.89950542e-016],
       [  1.00000000e+000,   7.80478134e-013],
       [  3.17501339e-038,   1.00000000e+000],
       [  0.00000000e+000,   1.00000000e+000],
       [  0.00000000e+000,   1.00000000e+000],
       [  1.85176775e-084,   1.00000000e+000],
       [  9.89259214e-001,   1.07407861e-002],
       [  9.96187913e-068,   1.00000000e+000],
       [  1.00000000e+000,   2.49956149e-016],
       [  1.30869517e-058,   1.00000000e+000],
       [  1.00000000e+000,   4.65401545e-014],
       [  1.00000000e+000,   5.81369075e-016],
       [  9.94060981e-001,   5.93901904e-003],
       [  1.79719349e-017,   1.00000000e+000],
       [  1.00000000e+000,   1.10920158e-014],
       [  1.3

In [9]:
lcf = LogisticRegression()
lcf.fit(X_train, y_train)
y_pred_lcf = lcf.predict(X_test)
y_prob_lcf = lcf.predict_proba(X_test)
print('Accuracy: ',accuracy_score(y_pred_lcf, y_test))

Accuracy:  0.972972972973


In [10]:
from sklearn.ensemble import IsolationForest
icf=IsolationForest(contamination=0.0017234102419808666,random_state=13)
icf.fit(X_train, y_train)
y_pred_icf = icf.predict(X_test)
#y_prob_icf = icf.predict_proba(X_test)



In [11]:

y_pred_icf[y_pred_icf == 1] = 0
y_pred_icf[y_pred_icf == -1] =1

In [12]:
 print('Accuracy: ',accuracy_score(y_pred_icf, y_test))

Accuracy:  0.527027027027


In [13]:
y_prob = (0.26148113552*y_prob_lcf+0.191034210122*y_prob_gcf+0.2828419442*y_prob_knn+0.264642710167*y_prob_svm)

In [14]:
print(y_prob)

[[  3.92978878e-03   9.96070211e-01]
 [  3.73145715e-03   9.96268543e-01]
 [  9.94931165e-01   5.06883518e-03]
 [  3.87194217e-03   9.96128058e-01]
 [  7.87049206e-01   2.12950794e-01]
 [  4.00419699e-03   9.95995803e-01]
 [  9.96644809e-01   3.35519115e-03]
 [  9.91821603e-01   8.17839655e-03]
 [  2.35467542e-02   9.76453246e-01]
 [  3.81401768e-03   9.96185982e-01]
 [  3.87220473e-03   9.96127795e-01]
 [  2.13437977e-03   9.97865620e-01]
 [  9.95437878e-01   4.56212214e-03]
 [  1.09085770e-03   9.98909142e-01]
 [  9.89421794e-01   1.05782060e-02]
 [  4.62698147e-06   9.99995373e-01]
 [  9.97708083e-01   2.29191697e-03]
 [  9.98095883e-01   1.90411730e-03]
 [  2.46657869e-01   7.53342131e-01]
 [  5.20643293e-07   9.99999479e-01]
 [  9.31360352e-01   6.86396479e-02]
 [  7.24522077e-04   9.99275478e-01]
 [  3.49134481e-01   6.50865519e-01]
 [  3.94903393e-03   9.96050966e-01]
 [  3.00314077e-03   9.96996859e-01]
 [  9.95004441e-01   4.99555946e-03]
 [  9.96531068e-01   3.46893247e-03]
 

In [15]:
def printgf(i,j):
    if (i>j):
        return 0
    else :return 1

In [16]:
y_pred=[printgf(i,j) for i,j in y_prob]

In [17]:
print('Accuracy: ',accuracy_score(y_pred, y_test))

Accuracy:  0.966216216216


In [18]:
def maxvote(y_pred_lcf,y_pred_gcf,y_pred_knn,y_pred_svm):
    i=0
    j=0
    for k in [y_pred_lcf,y_pred_gcf,y_pred_knn,y_pred_svm]:
        if k ==0:
            i=i+1
        else:
            j=j+1
    if(i>j): return 0
    else: return 1

In [19]:
y_pred2=[]
for i in range(len(y_pred_lcf)):
    y_pred2.append(maxvote(y_pred_lcf[i],y_pred_gcf[i],y_pred_knn[i],y_pred_svm[i]))

In [20]:
print('Accuracy: ',accuracy_score(y_pred2, y_test))

Accuracy:  0.972972972973


In [21]:
print('Accuracy: ',accuracy_score([1 for i in range(len(y_test))], y_test))

Accuracy:  0.472972972973


In [22]:
y_test

array([1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1], dtype=int64)